## 引入密鑰

In [11]:
from dotenv import load_dotenv
import os

# 載入 .env 檔案
load_dotenv()
# 設定 OpenAI API 密鑰
api_key=os.getenv("OPENAI_API_KEY")

## langchain
- 查看使用不同model：https://python.langchain.com/docs/integrations/chat/openai/

In [ ]:
!pip install langchain
!pip install langchain_openai

In [12]:
from langchain_openai import ChatOpenAI

# 初始化模型
model = ChatOpenAI(
    model="gpt-4",  # 指定模型名稱
    openai_api_key=api_key  # 傳入正確的 API 金鑰參數名稱
)

In [6]:
from langchain.schema import HumanMessage, SystemMessage
# 設置對話內容
messages = [
    SystemMessage(content="请你作为我的物理课助教，用通俗易懂且间接的语言帮我解释物理概念。"),
    HumanMessage(content="什么是波粒二象性？"),
]

In [8]:
response = model.invoke(messages)
response

AIMessage(content='想象你有一个超级能力，你可以既变成一个人，又可以变成一只鸟。当你是人的时候，你可以和别人交谈，抓东西，走路等等。当你变成鸟的时候，你就可以飞翔，鸣叫，吃虫子等等。你可以根据你的需要在人和鸟之间自由切换。\n\n这就是波粒二象性的描述。光（或其他粒子）就像这样的超级能力者，它有时候表现得像粒子，有时候又表现得像波动。当我们说光像粒子时，就像光能击中物体并使其发热，这就像一个人能够拿起和扔掉物体。而当我们说光像波动时，就像光能够通过水面产生波纹，这就像鸟儿在水面上飞翔留下的波纹。所以，光具有波粒二象性。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 296, 'prompt_tokens': 64, 'total_tokens': 360, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-78a2675b-5337-4d1a-871d-7a00aff695ee-0', usage_metadata={'input_tokens': 64, 'output_tokens': 296, 'total_tokens': 360, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## prompt template

In [13]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


- 使用SystemMessagePromptTemplate

In [16]:
system_template_text="您是一位專業的翻譯，能夠將{input_language}翻譯成{output_language}，並且輸出文本會根據用戶要求的任何語言風格進行調整。請只輸出翻譯後的文本，不要有任何其他內容。"
system_prompt_template = SystemMessagePromptTemplate.from_template(system_template_text)
print(system_prompt_template)
print(system_prompt_template.input_variables)
print()
system_prompt = system_prompt_template.format(input_language="英语", output_language="汉语")
print(system_prompt)

prompt=PromptTemplate(input_variables=['input_language', 'output_language'], input_types={}, partial_variables={}, template='您是一位專業的翻譯，能夠將{input_language}翻譯成{output_language}，並且輸出文本會根據用戶要求的任何語言風格進行調整。請只輸出翻譯後的文本，不要有任何其他內容。') additional_kwargs={}
['input_language', 'output_language']

content='您是一位專業的翻譯，能夠將英语翻譯成汉语，並且輸出文本會根據用戶要求的任何語言風格進行調整。請只輸出翻譯後的文本，不要有任何其他內容。' additional_kwargs={} response_metadata={}


- 使用HumanMessagePromptTemplate

In [17]:
human_template_text = "文本：{text}\n語言風格：{style}"
human_prompt_template = HumanMessagePromptTemplate.from_template(human_template_text)
print(human_prompt_template.input_variables)
print()
human_prompt = human_prompt_template.format(text="I'm so hungry I could eat a horse", style="文言文")
print(human_prompt)

['style', 'text']

content="文本：I'm so hungry I could eat a horse\n語言風格：文言文" additional_kwargs={} response_metadata={}


In [19]:
from langchain_openai import ChatOpenAI

# 初始化模型
model = ChatOpenAI(
    model="gpt-4",  # 指定模型名稱
    openai_api_key=api_key  # 傳入正確的 API 金鑰參數名稱
)

response = model.invoke([
    system_prompt,
    human_prompt
])

print(response.content)


我饥若狼，几乎可食马也。


## 在prompt裡面塞小範例

In [21]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

In [22]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "格式化以下客户信息：\n姓名 -> {customer_name}\n年龄 -> {customer_age}\n 城市 -> {customer_city}"),
        ("ai", "##客户信息\n- 客户姓名：{formatted_name}\n- 客户年龄：{formatted_age}\n- 客户所在地：{formatted_city}")
    ]
)

In [23]:
examples = [
    {
        "customer_name": "张三", 
        "customer_age": "27",
        "customer_city": "长沙",
        "formatted_name": "张三",
        "formatted_age": "27岁",
        "formatted_city": "湖南省长沙市"
    },
    {
        "customer_name": "李四", 
        "customer_age": "42",
        "customer_city": "广州",
        "formatted_name": "李四",
        "formatted_age": "42岁",
        "formatted_city": "广东省广州市"
    },
]

In [24]:
# 組成提示句
few_shot_template = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [25]:
# 叫他依照我鑰的格式去生成
final_prompt_template = ChatPromptTemplate.from_messages(
    [
        few_shot_template,
        ("human", "{input}"),
    ]
)

final_prompt = final_prompt_template.invoke({"input": "格式化以下客户信息：\n姓名 -> 王五\n年龄 -> 31\n 城市 -> 郑州'"})
final_prompt.messages


[HumanMessage(content='格式化以下客户信息：\n姓名 -> 张三\n年龄 -> 27\n 城市 -> 长沙', additional_kwargs={}, response_metadata={}),
 AIMessage(content='##客户信息\n- 客户姓名：张三\n- 客户年龄：27岁\n- 客户所在地：湖南省长沙市', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='格式化以下客户信息：\n姓名 -> 李四\n年龄 -> 42\n 城市 -> 广州', additional_kwargs={}, response_metadata={}),
 AIMessage(content='##客户信息\n- 客户姓名：李四\n- 客户年龄：42岁\n- 客户所在地：广东省广州市', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="格式化以下客户信息：\n姓名 -> 王五\n年龄 -> 31\n 城市 -> 郑州'", additional_kwargs={}, response_metadata={})]

In [26]:
# 初始化模型
model = ChatOpenAI(
    model="gpt-4",  # 指定模型名稱
    openai_api_key=api_key  # 傳入正確的 API 金鑰參數名稱
)

response = model.invoke(final_prompt)
print(response.content)

## 客户信息
- 客户姓名：王五
- 客户年龄：31岁
- 客户所在地：河南省郑州市
